In [7]:
import os

path = os.path.abspath("../etl/Test2")
print(f"Absolute path: {path}")
print(f"Path exists: {os.path.exists(path)}")

if os.path.exists(path):
    print("Files in the directory:")
    print(os.listdir(path))
else:
    print("The specified path does not exist.")

Absolute path: /opt/glue/scripts/etl/Test2
Path exists: True
Files in the directory:
['main_script.py', 'mastering.py', 'preprocessing.py', 'staging.py', 'utils.py']


In [8]:
import sys
import os

# Add the directory containing main_script.py to the system path
sys.path.append(os.path.abspath("../etl/Test2"))

from utils import get_spark_session

spark = get_spark_session("/opt/glue/jars/hadoop-aws-3.2.0.jar", "/opt/glue/jars/aws-java-sdk-bundle-1.11.375.jar")
if spark:
    print("Spark session initialized successfully")
else:
    print("Failed to initialize Spark session")


INFO:utils:Checking JAR paths: /opt/glue/jars/hadoop-aws-3.2.0.jar, /opt/glue/jars/aws-java-sdk-bundle-1.11.375.jar
INFO:utils:Spark session created successfully


Spark session initialized successfully


In [9]:
import staging
from utils import get_spark_session

spark = get_spark_session("/opt/glue/jars/hadoop-aws-3.2.0.jar", "/opt/glue/jars/aws-java-sdk-bundle-1.11.375.jar")
staging.stage_data(spark, "s3a://raw-data/", "s3a://staging-data/", "orders")


INFO:utils:Checking JAR paths: /opt/glue/jars/hadoop-aws-3.2.0.jar, /opt/glue/jars/aws-java-sdk-bundle-1.11.375.jar
INFO:utils:Spark session created successfully
INFO:staging:Staging data from s3a://raw-data/ to s3a://staging-data/ for table orders
INFO:staging:Data staging logic executed successfully


In [13]:
import preprocessing
from utils import get_spark_session

spark = get_spark_session("/opt/glue/jars/hadoop-aws-3.2.0.jar", "/opt/glue/jars/aws-java-sdk-bundle-1.11.375.jar")
preprocessing.preprocess_data(spark, "s3a://ecommerce-data-lake-730335322582-us-east-1-dev/02_staging/netsuite/", "s3a://ecommerce-data-lake-730335322582-us-east-1-dev/02_staging/netsuite/", "orders", "full_load")


INFO:utils:Checking JAR paths: /opt/glue/jars/hadoop-aws-3.2.0.jar, /opt/glue/jars/aws-java-sdk-bundle-1.11.375.jar
INFO:utils:Spark session created successfully
INFO:preprocessing:Preprocessing data for table: orders, process type: full_load
ERROR:preprocessing:Error preprocessing data for table orders: [UNABLE_TO_INFER_SCHEMA] Unable to infer schema for Parquet. It must be specified manually.


AnalysisException: [UNABLE_TO_INFER_SCHEMA] Unable to infer schema for Parquet. It must be specified manually.

In [11]:
import mastering
from utils import get_spark_session

spark = get_spark_session("/opt/glue/jars/hadoop-aws-3.2.0.jar", "/opt/glue/jars/aws-java-sdk-bundle-1.11.375.jar")
mastering.master_data(spark, "s3a://ecommerce-data-lake-730335322582-us-east-1-dev/03_preprocessing/netsuite/", "s3a://ecommerce-data-lake-730335322582-us-east-1-dev/04_master/", "orders")


INFO:utils:Checking JAR paths: /opt/glue/jars/hadoop-aws-3.2.0.jar, /opt/glue/jars/aws-java-sdk-bundle-1.11.375.jar
INFO:utils:Spark session created successfully
INFO:mastering:Mastering data for table: orders
24/06/28 22:23:45 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: s3a://preprocessed-data/orders/.
java.nio.file.AccessDeniedException: s3a://preprocessed-data/orders: getFileStatus on s3a://preprocessed-data/orders: com.amazonaws.services.s3.model.AmazonS3Exception: Forbidden (Service: Amazon S3; Status Code: 403; Error Code: 403 Forbidden; Request ID: 1YGD67J22F1T6DTX; S3 Extended Request ID: Z9HIuWtPDS1AnV4iNZ1qV1K5yJvpmkmaC5VMGneH/Wnbopurp+W0S/iRxEovdO0+MoRwso3PoAQ=), S3 Extended Request ID: Z9HIuWtPDS1AnV4iNZ1qV1K5yJvpmkmaC5VMGneH/Wnbopurp+W0S/iRxEovdO0+MoRwso3PoAQ=:403 Forbidden
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:230)
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUti

Py4JJavaError: An error occurred while calling o50.parquet.
: java.nio.file.AccessDeniedException: s3a://preprocessed-data/orders: getFileStatus on s3a://preprocessed-data/orders: com.amazonaws.services.s3.model.AmazonS3Exception: Forbidden (Service: Amazon S3; Status Code: 403; Error Code: 403 Forbidden; Request ID: 1YG4DCWA998PZEXD; S3 Extended Request ID: c34KjTLhZ3AfRdWHj1OT1x7n9XtSlIK1RlUlBHE1IGu7zicSvfgu7RCoqcMyVbR5Fnt5sQaIuEo=), S3 Extended Request ID: c34KjTLhZ3AfRdWHj1OT1x7n9XtSlIK1RlUlBHE1IGu7zicSvfgu7RCoqcMyVbR5Fnt5sQaIuEo=:403 Forbidden
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:230)
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:151)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.s3GetFileStatus(S3AFileSystem.java:2198)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.innerGetFileStatus(S3AFileSystem.java:2163)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.getFileStatus(S3AFileSystem.java:2102)
	at org.apache.hadoop.fs.FileSystem.exis